In [20]:
# Import of the packages
import numpy as np
import scipy.optimize as optimize
import matplotlib.pyplot as plt
import math
import scipy.stats as sts

In [21]:
class ParameterValueError(Exception):
    """ Custom exception raised when a parameter has an invalid value."""
    pass

In [421]:
# RVM class
class RVM:
    """ Relevance Vector Machine (RVM)
    
    Implementation of RVM for both regression and classification.
    
    Attributes:
        kerType: A string of the type of the desired kernel.
        rvmType: A string denoting the RVM type to be used. The string "EM" denotes
            an EM-like algorithm will be used to estimate the hyperparameters while
            the string "DD" denotes the direct differentiation approach.
        p: Integer value denoting the degree of the polynomial kernel.
        sigma: Float value denoting the smoothing factor of the Gaussian kernel.
        kappa: Float value denoting the scaling parameter of the sigmoid kernel.
        delta: Float value denoting the translation parameter of the sigmoid kernel.
        bTrained: boolean value which becomes true once the RVM has been trained.
    """

    EPSILON = 1e-10
    maxEpochs = 10000
    
    def __init__(self, kerType = 'poly', rvmType = "EM", p = 1, sigma = 1, 
                 kappa = 1, delta = 1):
        """ Initializes the RVM class (constructor).
        
            Raises:
                ParameterValueError: An error occured because a parameter had an
                    invalid value.  
        """
        # Check if the kernel type chosen is valid
        kerTypes = ['linear', 'poly', 'radial', 'sigmoid']
        if kerType not in kerTypes:
            raise ParameterValueError("ParameterValueError: The string " + kerType + \
                                       " does not denote a valid kernel type")
        # Check if the string denoting the rvmType has a valid value
        if rvmType != 'EM' and rvmType != 'DD':
            raise ParameterValueError('ParameterValueError: ' + rvmType, \
                                       " is not a valid RVM type value. Enter 'EM' or 'DD' as a value. ")
       
        self.kerType = kerType
        self.rvmType = rvmType
        self.p = p
        self.sigma = sigma
        self.kappa = kappa
        self.delta = delta
        self.bTrained = False

    def kernel(self, x, y):
        """ Kernel computation.
        
        It computes the kernel value based on the dot product
        between two vectors.
        
        Args:
            x: Input vector.
            y: Other input vector.
            
        Returns:
            The computed kernel value.
        """  
        if self.kerType == "linear":
            k = np.dot(x,y) + 1
        elif self.kerType == "poly":
            k = (np.dot(x,y) + 1) ** self.p
        elif self.kerType == "radial":
            k = math.exp(-(np.dot(x-y,x-y))/(2*self.sigma))
        elif self.kerType == "sigmoid":
            k = math.atanh(self.kappa * np.dot(x,y) - self.delta)

        return k
    
    def getKernelMatrix(self, X):
        """ Evaluates the kernel matrix K given a set of input samples.

            Args:
                X_tr: A NxM matrix with a M dimensional training input sample
                    in each row.
                
            Returns:
                An NxN Kernel matrix where N is the number of input samples.
        """
        N = X.shape[0]
        K = np.zeros((N, N))
        for i in range(N):
            for j in range(N):
                K[i,j] = self.kernel(X[i], X[j])
        return K

    def getGammaValues(self, alpha_values, Sigma):
        """Evaluates the gamma values.
        
            Args:
                alpha_values: N-dimensional vector with the hyperparameters of
                    the marginal likelihood.
                Sigma: NxN covariance matrix of the posterior
                
            Returns: A N-dimensional vector with the gamma values where 
                gamma_values[i] = 1 - alpha_values[i] * Sigma[i][i]
        """
        N = alpha_values.shape[0]
        gamma_values = np.zeros(N)
        for i in range(N):
            gamma_values[i] = 1 - alpha_values[i] * Sigma[i][i]
        return gamma_values
        
    def train(self, X_tr, Y_tr):
        """ RVM training method
        
        Applies an EM-like algorithm or direct differentiation to estimate the
        optimal hyperparameters (alpha and sigma) needed to make predictions
        using the marginal likelihood.
        
        
        Args:
            X_tr: A matrix with a training input sample in each row.
            Y_tr: A vector with the output values of each input sample
                in X_tr.
        
        Returns:
            None
        """
        # Get number of training data samples
        N = X_tr.shape[0]
        # Initialize the alpha values (weight precision values) and the A matrix
        alpha_values = np.ones(N + 1)
        A = np.diag(alpha_values)
        # Initialize the sigma squared value and the B matrix
        sigma_squared = 1
        B = np.identity(N) * (sigma_squared ** -1)
        # Calculate kernel matrix K and append a column with ones in the front (for the bias term??)
        K = self.getKernelMatrix(X_tr)        
        K = np.hstack((np.ones(N).reshape((N, 1)), K))
        # Calculate Sigma and mu based on the initialized parameters
        Sigma = np.linalg.inv(K.T.dot(B).dot(K) + A)
        mu = Sigma.dot(K.T).dot(B).dot(Y_tr)
        # Calculate initial gamma values
        gamma_values = self.getGammaValues(alpha_values, Sigma)
        
        

        # Approximate optimal hyperparameter values iteratively
        next_alpha_values = np.zeros(N + 1)
        for epoch in range(self.maxEpochs):
            # Evaluate alpha values
            for i in range(N + 1):
                if self.rvmType == "EM":
                    next_alpha_values[i] = 1 / (Sigma[i][i] + mu[i] ** 2)
                elif self.rvmType == "DD":
                    pass                
            # Evaluate sigma value
            next_sigma_squared = (np.linalg.norm(Y_tr - K.dot(mu)) ** 2) / (N - np.sum(gamma_values))
            # Check if algorithm has converged
            if (np.sum(np.absolute(next_alpha_values - alpha_values)) < self.EPSILON and
                abs(next_sigma_squared - sigma_squared) < self.EPSILON):
                    break
            # If algorithm has not converged, update all the variables
            alpha_values = next_alpha_values
            sigma_squared = next_sigma_squared
            A = np.diag(alpha_values)
            B = np.identity(N) * (sigma_squared ** -1)
            Sigma = np.linalg.inv(K.T.dot(B).dot(K) + A)
            mu = Sigma.dot(K.T).dot(B).dot(Y_tr)
            gamma_values = self.getGammaValues(alpha_values, Sigma)
            print("EPOCH: " + str(epoch))
        print(alpha_values)
        

        

In [422]:
N_tr = 100
N_ts = 1000

def trans(x):
    N = np.shape(x)[0]
    y = np.sin(x) + 3 + np.random.normal(loc=0, scale=0.1, size=N)
    return y 

x_tr = np.linspace(0,7.5,num=N_tr)
y_tr = trans(x_tr)
x_ts = np.linspace(0,7.5,num=N_ts)
y_ts = trans(x_ts)

In [423]:
try:
    s = RVM(kerType = "radial", sigma = 10, rvmType = "EM")
    s.train(x_tr, y_tr)
except Exception as e:
    print(e)
    

EPOCH: 0
EPOCH: 1
EPOCH: 2
EPOCH: 3
EPOCH: 4
EPOCH: 5
EPOCH: 6
EPOCH: 7
EPOCH: 8
EPOCH: 9
EPOCH: 10
EPOCH: 11
EPOCH: 12
EPOCH: 13
EPOCH: 14
EPOCH: 15
EPOCH: 16
EPOCH: 17
EPOCH: 18
EPOCH: 19
EPOCH: 20
EPOCH: 21
EPOCH: 22
EPOCH: 23
EPOCH: 24
EPOCH: 25
EPOCH: 26
EPOCH: 27
EPOCH: 28
EPOCH: 29
EPOCH: 30
EPOCH: 31
EPOCH: 32
EPOCH: 33
EPOCH: 34
EPOCH: 35
EPOCH: 36
EPOCH: 37
EPOCH: 38
EPOCH: 39
EPOCH: 40
EPOCH: 41
EPOCH: 42
EPOCH: 43
EPOCH: 44
EPOCH: 45
EPOCH: 46
EPOCH: 47
EPOCH: 48
EPOCH: 49
EPOCH: 50
EPOCH: 51
EPOCH: 52
EPOCH: 53
EPOCH: 54
EPOCH: 55
EPOCH: 56
EPOCH: 57
EPOCH: 58
EPOCH: 59
EPOCH: 60
EPOCH: 61
EPOCH: 62
EPOCH: 63
EPOCH: 64
EPOCH: 65
EPOCH: 66
EPOCH: 67
EPOCH: 68
EPOCH: 69
EPOCH: 70
EPOCH: 71
EPOCH: 72
EPOCH: 73
EPOCH: 74
EPOCH: 75
EPOCH: 76
EPOCH: 77
EPOCH: 78
EPOCH: 79
EPOCH: 80
EPOCH: 81
EPOCH: 82
EPOCH: 83
EPOCH: 84
EPOCH: 85
EPOCH: 86
EPOCH: 87
EPOCH: 88
EPOCH: 89
EPOCH: 90
EPOCH: 91
EPOCH: 92
EPOCH: 93
EPOCH: 94
EPOCH: 95
EPOCH: 96
EPOCH: 97
EPOCH: 98
EPOCH: 99
EPOCH: 100

EPOCH: 841
EPOCH: 842
EPOCH: 843
EPOCH: 844
EPOCH: 845
EPOCH: 846
EPOCH: 847
EPOCH: 848
EPOCH: 849
EPOCH: 850
EPOCH: 851
EPOCH: 852
EPOCH: 853
EPOCH: 854
EPOCH: 855
EPOCH: 856
EPOCH: 857
EPOCH: 858
EPOCH: 859
EPOCH: 860
EPOCH: 861
EPOCH: 862
EPOCH: 863
EPOCH: 864
EPOCH: 865
EPOCH: 866
EPOCH: 867
EPOCH: 868
EPOCH: 869
EPOCH: 870
EPOCH: 871
EPOCH: 872
EPOCH: 873
EPOCH: 874
EPOCH: 875
EPOCH: 876
EPOCH: 877
EPOCH: 878
EPOCH: 879
EPOCH: 880
EPOCH: 881
EPOCH: 882
EPOCH: 883
EPOCH: 884
EPOCH: 885
EPOCH: 886
EPOCH: 887
EPOCH: 888
EPOCH: 889
EPOCH: 890
EPOCH: 891
EPOCH: 892
EPOCH: 893
EPOCH: 894
EPOCH: 895
EPOCH: 896
EPOCH: 897
EPOCH: 898
EPOCH: 899
EPOCH: 900
EPOCH: 901
EPOCH: 902
EPOCH: 903
EPOCH: 904
EPOCH: 905
EPOCH: 906
EPOCH: 907
EPOCH: 908
EPOCH: 909
EPOCH: 910
EPOCH: 911
EPOCH: 912
EPOCH: 913
EPOCH: 914
EPOCH: 915
EPOCH: 916
EPOCH: 917
EPOCH: 918
EPOCH: 919
EPOCH: 920
EPOCH: 921
EPOCH: 922
EPOCH: 923
EPOCH: 924
EPOCH: 925
EPOCH: 926
EPOCH: 927
EPOCH: 928
EPOCH: 929
EPOCH: 930
EPOCH: 931

EPOCH: 1639
EPOCH: 1640
EPOCH: 1641
EPOCH: 1642
EPOCH: 1643
EPOCH: 1644
EPOCH: 1645
EPOCH: 1646
EPOCH: 1647
EPOCH: 1648
EPOCH: 1649
EPOCH: 1650
EPOCH: 1651
EPOCH: 1652
EPOCH: 1653
EPOCH: 1654
EPOCH: 1655
EPOCH: 1656
EPOCH: 1657
EPOCH: 1658
EPOCH: 1659
EPOCH: 1660
EPOCH: 1661
EPOCH: 1662
EPOCH: 1663
EPOCH: 1664
EPOCH: 1665
EPOCH: 1666
EPOCH: 1667
EPOCH: 1668
EPOCH: 1669
EPOCH: 1670
EPOCH: 1671
EPOCH: 1672
EPOCH: 1673
EPOCH: 1674
EPOCH: 1675
EPOCH: 1676
EPOCH: 1677
EPOCH: 1678
EPOCH: 1679
EPOCH: 1680
EPOCH: 1681
EPOCH: 1682
EPOCH: 1683
EPOCH: 1684
EPOCH: 1685
EPOCH: 1686
EPOCH: 1687
EPOCH: 1688
EPOCH: 1689
EPOCH: 1690
EPOCH: 1691
EPOCH: 1692
EPOCH: 1693
EPOCH: 1694
EPOCH: 1695
EPOCH: 1696
EPOCH: 1697
EPOCH: 1698
EPOCH: 1699
EPOCH: 1700
EPOCH: 1701
EPOCH: 1702
EPOCH: 1703
EPOCH: 1704
EPOCH: 1705
EPOCH: 1706
EPOCH: 1707
EPOCH: 1708
EPOCH: 1709
EPOCH: 1710
EPOCH: 1711
EPOCH: 1712
EPOCH: 1713
EPOCH: 1714
EPOCH: 1715
EPOCH: 1716
EPOCH: 1717
EPOCH: 1718
EPOCH: 1719
EPOCH: 1720
EPOCH: 1721
EPOC

EPOCH: 2434
EPOCH: 2435
EPOCH: 2436
EPOCH: 2437
EPOCH: 2438
EPOCH: 2439
EPOCH: 2440
EPOCH: 2441
EPOCH: 2442
EPOCH: 2443
EPOCH: 2444
EPOCH: 2445
EPOCH: 2446
EPOCH: 2447
EPOCH: 2448
EPOCH: 2449
EPOCH: 2450
EPOCH: 2451
EPOCH: 2452
EPOCH: 2453
EPOCH: 2454
EPOCH: 2455
EPOCH: 2456
EPOCH: 2457
EPOCH: 2458
EPOCH: 2459
EPOCH: 2460
EPOCH: 2461
EPOCH: 2462
EPOCH: 2463
EPOCH: 2464
EPOCH: 2465
EPOCH: 2466
EPOCH: 2467
EPOCH: 2468
EPOCH: 2469
EPOCH: 2470
EPOCH: 2471
EPOCH: 2472
EPOCH: 2473
EPOCH: 2474
EPOCH: 2475
EPOCH: 2476
EPOCH: 2477
EPOCH: 2478
EPOCH: 2479
EPOCH: 2480
EPOCH: 2481
EPOCH: 2482
EPOCH: 2483
EPOCH: 2484
EPOCH: 2485
EPOCH: 2486
EPOCH: 2487
EPOCH: 2488
EPOCH: 2489
EPOCH: 2490
EPOCH: 2491
EPOCH: 2492
EPOCH: 2493
EPOCH: 2494
EPOCH: 2495
EPOCH: 2496
EPOCH: 2497
EPOCH: 2498
EPOCH: 2499
EPOCH: 2500
EPOCH: 2501
EPOCH: 2502
EPOCH: 2503
EPOCH: 2504
EPOCH: 2505
EPOCH: 2506
EPOCH: 2507
EPOCH: 2508
EPOCH: 2509
EPOCH: 2510
EPOCH: 2511
EPOCH: 2512
EPOCH: 2513
EPOCH: 2514
EPOCH: 2515
EPOCH: 2516
EPOC

EPOCH: 3196
EPOCH: 3197
EPOCH: 3198
EPOCH: 3199
EPOCH: 3200
EPOCH: 3201
EPOCH: 3202
EPOCH: 3203
EPOCH: 3204
EPOCH: 3205
EPOCH: 3206
EPOCH: 3207
EPOCH: 3208
EPOCH: 3209
EPOCH: 3210
EPOCH: 3211
EPOCH: 3212
EPOCH: 3213
EPOCH: 3214
EPOCH: 3215
EPOCH: 3216
EPOCH: 3217
EPOCH: 3218
EPOCH: 3219
EPOCH: 3220
EPOCH: 3221
EPOCH: 3222
EPOCH: 3223
EPOCH: 3224
EPOCH: 3225
EPOCH: 3226
EPOCH: 3227
EPOCH: 3228
EPOCH: 3229
EPOCH: 3230
EPOCH: 3231
EPOCH: 3232
EPOCH: 3233
EPOCH: 3234
EPOCH: 3235
EPOCH: 3236
EPOCH: 3237
EPOCH: 3238
EPOCH: 3239
EPOCH: 3240
EPOCH: 3241
EPOCH: 3242
EPOCH: 3243
EPOCH: 3244
EPOCH: 3245
EPOCH: 3246
EPOCH: 3247
EPOCH: 3248
EPOCH: 3249
EPOCH: 3250
EPOCH: 3251
EPOCH: 3252
EPOCH: 3253
EPOCH: 3254
EPOCH: 3255
EPOCH: 3256
EPOCH: 3257
EPOCH: 3258
EPOCH: 3259
EPOCH: 3260
EPOCH: 3261
EPOCH: 3262
EPOCH: 3263
EPOCH: 3264
EPOCH: 3265
EPOCH: 3266
EPOCH: 3267
EPOCH: 3268
EPOCH: 3269
EPOCH: 3270
EPOCH: 3271
EPOCH: 3272
EPOCH: 3273
EPOCH: 3274
EPOCH: 3275
EPOCH: 3276
EPOCH: 3277
EPOCH: 3278
EPOC

...

EPOCH: 6235
EPOCH: 6236
EPOCH: 6237
EPOCH: 6238
EPOCH: 6239
EPOCH: 6240
EPOCH: 6241
EPOCH: 6242
EPOCH: 6243
EPOCH: 6244
EPOCH: 6245
EPOCH: 6246
EPOCH: 6247
EPOCH: 6248
EPOCH: 6249
EPOCH: 6250
EPOCH: 6251
EPOCH: 6252
EPOCH: 6253
EPOCH: 6254
EPOCH: 6255
EPOCH: 6256
EPOCH: 6257
EPOCH: 6258
EPOCH: 6259
EPOCH: 6260
EPOCH: 6261
EPOCH: 6262
EPOCH: 6263
EPOCH: 6264
EPOCH: 6265
EPOCH: 6266
EPOCH: 6267
EPOCH: 6268
EPOCH: 6269
EPOCH: 6270
EPOCH: 6271
EPOCH: 6272
EPOCH: 6273
EPOCH: 6274
EPOCH: 6275
EPOCH: 6276
EPOCH: 6277
EPOCH: 6278
EPOCH: 6279
EPOCH: 6280
EPOCH: 6281
EPOCH: 6282
EPOCH: 6283
EPOCH: 6284
EPOCH: 6285
EPOCH: 6286
EPOCH: 6287
EPOCH: 6288
EPOCH: 6289
EPOCH: 6290
EPOCH: 6291
EPOCH: 6292
EPOCH: 6293
EPOCH: 6294
EPOCH: 6295
EPOCH: 6296
EPOCH: 6297
EPOCH: 6298
EPOCH: 6299
EPOCH: 6300
EPOCH: 6301
EPOCH: 6302
EPOCH: 6303
EPOCH: 6304
EPOCH: 6305
EPOCH: 6306
EPOCH: 6307
EPOCH: 6308
EPOCH: 6309
EPOCH: 6310
EPOCH: 6311
EPOCH: 6312
EPOCH: 6313
EPOCH: 6314
EPOCH: 6315
EPOCH: 6316
EPOCH: 6317
EPOC

EPOCH: 6924
EPOCH: 6925
EPOCH: 6926
EPOCH: 6927
EPOCH: 6928
EPOCH: 6929
EPOCH: 6930
EPOCH: 6931
EPOCH: 6932
EPOCH: 6933
EPOCH: 6934
EPOCH: 6935
EPOCH: 6936
EPOCH: 6937
EPOCH: 6938
EPOCH: 6939
EPOCH: 6940
EPOCH: 6941
EPOCH: 6942
EPOCH: 6943
EPOCH: 6944
EPOCH: 6945
EPOCH: 6946
EPOCH: 6947
EPOCH: 6948
EPOCH: 6949
EPOCH: 6950
EPOCH: 6951
EPOCH: 6952
EPOCH: 6953
EPOCH: 6954
EPOCH: 6955
EPOCH: 6956
EPOCH: 6957
EPOCH: 6958
EPOCH: 6959
EPOCH: 6960
EPOCH: 6961
EPOCH: 6962
EPOCH: 6963
EPOCH: 6964
EPOCH: 6965
EPOCH: 6966
EPOCH: 6967
EPOCH: 6968
EPOCH: 6969
EPOCH: 6970
EPOCH: 6971
EPOCH: 6972
EPOCH: 6973
EPOCH: 6974
EPOCH: 6975
EPOCH: 6976
EPOCH: 6977
EPOCH: 6978
EPOCH: 6979
EPOCH: 6980
EPOCH: 6981
EPOCH: 6982
EPOCH: 6983
EPOCH: 6984
EPOCH: 6985
EPOCH: 6986
EPOCH: 6987
EPOCH: 6988
EPOCH: 6989
EPOCH: 6990
EPOCH: 6991
EPOCH: 6992
EPOCH: 6993
EPOCH: 6994
EPOCH: 6995
EPOCH: 6996
EPOCH: 6997
EPOCH: 6998
EPOCH: 6999
EPOCH: 7000
EPOCH: 7001
EPOCH: 7002
EPOCH: 7003
EPOCH: 7004
EPOCH: 7005
EPOCH: 7006
EPOC

EPOCH: 7733
EPOCH: 7734
EPOCH: 7735
EPOCH: 7736
EPOCH: 7737
EPOCH: 7738
EPOCH: 7739
EPOCH: 7740
EPOCH: 7741
EPOCH: 7742
EPOCH: 7743
EPOCH: 7744
EPOCH: 7745
EPOCH: 7746
EPOCH: 7747
EPOCH: 7748
EPOCH: 7749
EPOCH: 7750
EPOCH: 7751
EPOCH: 7752
EPOCH: 7753
EPOCH: 7754
EPOCH: 7755
EPOCH: 7756
EPOCH: 7757
EPOCH: 7758
EPOCH: 7759
EPOCH: 7760
EPOCH: 7761
EPOCH: 7762
EPOCH: 7763
EPOCH: 7764
EPOCH: 7765
EPOCH: 7766
EPOCH: 7767
EPOCH: 7768
EPOCH: 7769
EPOCH: 7770
EPOCH: 7771
EPOCH: 7772
EPOCH: 7773
EPOCH: 7774
EPOCH: 7775
EPOCH: 7776
EPOCH: 7777
EPOCH: 7778
EPOCH: 7779
EPOCH: 7780
EPOCH: 7781
EPOCH: 7782
EPOCH: 7783
EPOCH: 7784
EPOCH: 7785
EPOCH: 7786
EPOCH: 7787
EPOCH: 7788
EPOCH: 7789
EPOCH: 7790
EPOCH: 7791
EPOCH: 7792
EPOCH: 7793
EPOCH: 7794
EPOCH: 7795
EPOCH: 7796
EPOCH: 7797
EPOCH: 7798
EPOCH: 7799
EPOCH: 7800
EPOCH: 7801
EPOCH: 7802
EPOCH: 7803
EPOCH: 7804
EPOCH: 7805
EPOCH: 7806
EPOCH: 7807
EPOCH: 7808
EPOCH: 7809
EPOCH: 7810
EPOCH: 7811
EPOCH: 7812
EPOCH: 7813
EPOCH: 7814
EPOCH: 7815
EPOC

EPOCH: 8522
EPOCH: 8523
EPOCH: 8524
EPOCH: 8525
EPOCH: 8526
EPOCH: 8527
EPOCH: 8528
EPOCH: 8529
EPOCH: 8530
EPOCH: 8531
EPOCH: 8532
EPOCH: 8533
EPOCH: 8534
EPOCH: 8535
EPOCH: 8536
EPOCH: 8537
EPOCH: 8538
EPOCH: 8539
EPOCH: 8540
EPOCH: 8541
EPOCH: 8542
EPOCH: 8543
EPOCH: 8544
EPOCH: 8545
EPOCH: 8546
EPOCH: 8547
EPOCH: 8548
EPOCH: 8549
EPOCH: 8550
EPOCH: 8551
EPOCH: 8552
EPOCH: 8553
EPOCH: 8554
EPOCH: 8555
EPOCH: 8556
EPOCH: 8557
EPOCH: 8558
EPOCH: 8559
EPOCH: 8560
EPOCH: 8561
EPOCH: 8562
EPOCH: 8563
EPOCH: 8564
EPOCH: 8565
EPOCH: 8566
EPOCH: 8567
EPOCH: 8568
EPOCH: 8569
EPOCH: 8570
EPOCH: 8571
EPOCH: 8572
EPOCH: 8573
EPOCH: 8574
EPOCH: 8575
EPOCH: 8576
EPOCH: 8577
EPOCH: 8578
EPOCH: 8579
EPOCH: 8580
EPOCH: 8581
EPOCH: 8582
EPOCH: 8583
EPOCH: 8584
EPOCH: 8585
EPOCH: 8586
EPOCH: 8587
EPOCH: 8588
EPOCH: 8589
EPOCH: 8590
EPOCH: 8591
EPOCH: 8592
EPOCH: 8593
EPOCH: 8594
EPOCH: 8595
EPOCH: 8596
EPOCH: 8597
EPOCH: 8598
EPOCH: 8599
EPOCH: 8600
EPOCH: 8601
EPOCH: 8602
EPOCH: 8603
EPOCH: 8604
EPOC

EPOCH: 9292
EPOCH: 9293
EPOCH: 9294
EPOCH: 9295
EPOCH: 9296
EPOCH: 9297
EPOCH: 9298
EPOCH: 9299
EPOCH: 9300
EPOCH: 9301
EPOCH: 9302
EPOCH: 9303
EPOCH: 9304
EPOCH: 9305
EPOCH: 9306
EPOCH: 9307
EPOCH: 9308
EPOCH: 9309
EPOCH: 9310
EPOCH: 9311
EPOCH: 9312
EPOCH: 9313
EPOCH: 9314
EPOCH: 9315
EPOCH: 9316
EPOCH: 9317
EPOCH: 9318
EPOCH: 9319
EPOCH: 9320
EPOCH: 9321
EPOCH: 9322
EPOCH: 9323
EPOCH: 9324
EPOCH: 9325
EPOCH: 9326
EPOCH: 9327
EPOCH: 9328
EPOCH: 9329
EPOCH: 9330
EPOCH: 9331
EPOCH: 9332
EPOCH: 9333
EPOCH: 9334
EPOCH: 9335
EPOCH: 9336
EPOCH: 9337
EPOCH: 9338
EPOCH: 9339
EPOCH: 9340
EPOCH: 9341
EPOCH: 9342
EPOCH: 9343
EPOCH: 9344
EPOCH: 9345
EPOCH: 9346
EPOCH: 9347
EPOCH: 9348
EPOCH: 9349
EPOCH: 9350
EPOCH: 9351
EPOCH: 9352
EPOCH: 9353
EPOCH: 9354
EPOCH: 9355
EPOCH: 9356
EPOCH: 9357
EPOCH: 9358
EPOCH: 9359
EPOCH: 9360
EPOCH: 9361
EPOCH: 9362
EPOCH: 9363
EPOCH: 9364
EPOCH: 9365
EPOCH: 9366
EPOCH: 9367
EPOCH: 9368
EPOCH: 9369
EPOCH: 9370
EPOCH: 9371
EPOCH: 9372
EPOCH: 9373
EPOCH: 9374
EPOC